<a href="https://colab.research.google.com/github/frank-morales2020/Cloud_curious/blob/master/POC_LLM_HF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://huggingface.co/collections/open-llm-leaderboard/the-big-benchmarks-collection-64faca6335a7fc7d4ffe974a


https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration

In [ ]:
!nvidia-smi

Mon Apr 29 14:15:49 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0              45W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

https://www.engineering.com/story/design-and-engineering-performance-and-flexibility-the-nvidia-l4-tensor-core-gpu

In [ ]:
!pip install colab-env --quiet
!pip install tiktoken -q
!pip install accelerate -q

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 6.7 MB/s eta 0:00:00


In [ ]:
import colab_env
import os

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

from huggingface_hub import login

login(
  token=access_token_write,
  add_to_git_credential=True
)

Mounted at /content/gdrive
Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
!pip install torchtune -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.7/210.7 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 41.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.7/73.7 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 79.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 16.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
!tune -h

usage: tune [-h] {download,ls,cp,run,validate} ...

Welcome to the TorchTune CLI!

options:
  -h, --help            show this help message and exit

subcommands:
  {download,ls,cp,run,validate}
    download            Download a model from the Hugging Face Hub.
    ls                  List all built-in recipes and configs
    cp                  Copy a built-in recipe or config to a local path.
    run                 Run a recipe. For distributed recipes, this supports all torchrun
                        arguments.
    validate            Validate a config and ensure that it is well-formed.


In [ ]:
!tune download frankmorales2020/torchtune-Llama-2-7b --output-dir /tmp/Llama-2-7b-hf

In [ ]:
!ls -ltha /tmp/Llama-2-7b-hf/*

lrwxrwxrwx 1 root root  151 Apr 29 14:32 /tmp/Llama-2-7b-hf/hf_model_0001_0.pt -> ../../root/.cache/huggingface/hub/models--frankmorales2020--torchtune-Llama-2-7b/blobs/40be895e082a4f8b9d610326d2f9de309a58ae0219346b8afd608d27b0451c19
lrwxrwxrwx 1 root root  151 Apr 29 14:26 /tmp/Llama-2-7b-hf/pytorch_model-00001-of-00002.bin -> ../../root/.cache/huggingface/hub/models--frankmorales2020--torchtune-Llama-2-7b/blobs/ee62ed2ad7ded505ae47df50bc6c52916860dfb1c009df4715148cc4bfb50d2f
lrwxrwxrwx 1 root root  151 Apr 29 14:21 /tmp/Llama-2-7b-hf/pytorch_model-00002-of-00002.bin -> ../../root/.cache/huggingface/hub/models--frankmorales2020--torchtune-Llama-2-7b/blobs/1fd7762035b3ca4f2d6af6bf10129689a119b7c38058025f9842511532ea02fb
lrwxrwxrwx 1 root root  151 Apr 29 14:21 /tmp/Llama-2-7b-hf/hf_model_0002_0.pt -> ../../root/.cache/huggingface/hub/models--frankmorales2020--torchtune-Llama-2-7b/blobs/f93037a0b82968e3c11e26de864988d642f37154bdfd1f314630c00d06ab4b17
-rw-r--r-- 1 root root 489K Apr 29 1

In [ ]:
!tune cp generation /content/custom_generation_config.yaml

Copied file to /content/custom_generation_config.yaml


before fine-tunne

In [ ]:
!tune run generate --config /content/custom_generation_config.yaml prompt="What are some interesting sites to visit in the Bay Area?"

INFO:torchtune.utils.logging:Running InferenceRecipe with resolved config:

checkpointer:
  _component_: torchtune.utils.FullModelHFCheckpointer
  checkpoint_dir: /tmp/Llama-2-7b-hf/
  checkpoint_files:
  - pytorch_model-00001-of-00002.bin
  - pytorch_model-00002-of-00002.bin
  model_type: LLAMA2
  output_dir: /tmp/Llama-2-7b-hf/
device: cuda
dtype: bf16
max_new_tokens: 300
model:
  _component_: torchtune.models.llama2.llama2_7b
prompt: What are some interesting sites to visit in the Bay Area?
quantizer: null
seed: 1234
temperature: 0.6
tokenizer:
  _component_: torchtune.models.llama2.llama2_tokenizer
  path: /tmp/Llama-2-7b-hf/tokenizer.model
top_k: 300

DEBUG:torchtune.utils.logging:Setting manual seed to local seed 1234. Local seed is seed + rank = 1234 + 0
INFO:torchtune.utils.logging:Model is initialized with precision torch.bfloat16.
INFO:torchtune.utils.logging:What are some interesting sites to visit in the Bay Area?
I'm going to be spending the weekend in San Francisco and wo

fine-tune model

checkpoint_files: [
        hf_model_0001_0.pt,
        hf_model_0002_0.pt,
  ]

In [ ]:
!tune run generate --config /content/custom_generation_config.yaml prompt="What are some interesting sites to visit in the Bay Area?"

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import torch

#model_id = "davidkim205/Rhea-72b-v0.5" # HF AVERAGE = 81.22 #1 APRIL 5TH, 2024

model_id = "mistralai/Mixtral-8x22B-Instruct-v0.1"

model_id = "meta-llama/Llama-2-7b-hf" #29/04/2024

#Metric	Value
#Avg.	81.22
#AI2 Reasoning Challenge (25-Shot)	79.78
#HellaSwag (10-Shot)	91.15
#MMLU (5-Shot)	77.95
#TruthfulQA (0-shot)	74.50
#Winogrande (5-shot)	87.85
#GSM8k (5-shot)	76.12


tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", torch_dtype=torch.bfloat16, trust_remote_code=True, do_sample=True)

# load into pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [ ]:
#prompt="What was the first album Beyoncé released as a solo artist?"
prompt="What is the capital of canada?"
outputs = pipe(prompt, max_new_tokens=512, do_sample=True, temperature=0.8,top_k=50, eos_token_id=pipe.tokenizer.eos_token_id, pad_token_id=pipe.tokenizer.eos_token_id)

In [ ]:
print('Question: %s'%prompt)
print(f"Generated Answer:\n{outputs[0]['generated_text'][len(prompt):].strip()}")